In [1]:
from nilearn.input_data import NiftiLabelsMasker, NiftiMasker
from nilearn import datasets, image, masking
import numpy as np
import nibabel as nib


In [5]:
Y = np.load('/Volumes/LaCie/EPFL/Mastersem3/SemesterProjectND/Y/sub2/After_The_Rain/Y_voxel_time_series.npy')
Y.shape

(555, 216990)

In [3]:
anat_img = image.load_img("/Volumes/LaCie/EPFL/Mastersem3/SemesterProjectND/irmf/derivatives/preprocessing/sub-S02/ses-2/anat/sub-S02_ses-2_space-subject_desc-anat_bold.nii.gz")


In [ ]:
""" 
# Load an atlas : This fetches the Harvard-Oxford cortical atlas, which divides the brain into regions of interest (ROIs).
Key output:
atlas.maps: A NIfTI file with labeled brain regions (each voxel is assigned a region number).
atlas.labels: A list of region names corresponding to the region numbers.
Example:
Region 1 = Frontal Pole, Region 2 = Insular Cortex, etc.
"""

atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-1mm')


# Initialize the masker : This masker uses the atlas to identify voxels that belong to regions in question
# standardize = True : Normalizes the time series within each region to have a mean of 0 and a standard deviation of 1. 
# This removes baseline signal differences and improves comparability between regions.
masker = NiftiLabelsMasker(labels_img=atlas.maps, standardize=True)


### All of this is done for 

# Extract regional time-series
region_time_series = masker.fit_transform('smoothed_fmri.nii.gz')

# There are different ways to take our matrix Y 

## 1) Take the matrix Y according to different regions of the brain 

### The result is a time series matrix with shape : (n time points, n regions)
An atlas in the context of neuroimaging is a standardized reference map of the brain that divides it into regions or parcels based on anatomical, functional, or statistical criteria. These regions are used to label and analyze specific areas of the brain in a consistent and interpretable way.


The number of regions corresponds to the number of labels in the atlas data 


TR (Repetition time) = 1.9 s
- In order to sync the fmri data ($Y$) with the features (design matrix) ($X$, i.e : $X_1,...X_n$ if n features), the fmri data must be set 4 TRs after the movie frame is set. 

TE (Echo Time) = 2.27ms

Flip Angle = 9 degrees is the angle at which the RF pulse tips the magnetic moment of protons in the tissue. It influences signal intensity and the amount of tissue contrast.

In [ ]:
# Each row of the region_time_series corresponds to the BOLD signal for all regions at a particular time point.
# Each column represents the BOLD signal for a specific brain region defined by the atlas you applied 
#       (e.g., the Harvard-Oxford cortical atlas).

region_time_series.shape

(528, 48)

In [7]:
# Print region names and number of regions
print(f"Number of regions: {len(atlas.labels)}")
print("Regions:", atlas.labels)

Number of regions: 49
Regions: ['Background', 'Frontal Pole', 'Insular Cortex', 'Superior Frontal Gyrus', 'Middle Frontal Gyrus', 'Inferior Frontal Gyrus, pars triangularis', 'Inferior Frontal Gyrus, pars opercularis', 'Precentral Gyrus', 'Temporal Pole', 'Superior Temporal Gyrus, anterior division', 'Superior Temporal Gyrus, posterior division', 'Middle Temporal Gyrus, anterior division', 'Middle Temporal Gyrus, posterior division', 'Middle Temporal Gyrus, temporooccipital part', 'Inferior Temporal Gyrus, anterior division', 'Inferior Temporal Gyrus, posterior division', 'Inferior Temporal Gyrus, temporooccipital part', 'Postcentral Gyrus', 'Superior Parietal Lobule', 'Supramarginal Gyrus, anterior division', 'Supramarginal Gyrus, posterior division', 'Angular Gyrus', 'Lateral Occipital Cortex, superior division', 'Lateral Occipital Cortex, inferior division', 'Intracalcarine Cortex', 'Frontal Medial Cortex', 'Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)', 'Subc

In [ ]:


# Save the region_time_series array
np.save('region_time_series.npy', region_time_series)

region_time_series2 = np.load('region_time_series.npy')


## 2) Take the matrix Y according to voxels as Individual Observations (Voxel-level Analysis)

### a) Generate a brain mask

In [ ]:

# Load your anatomical MRI image (replace the file path as needed)
anat_img = image.load_img('/Volumes/LaCie/EPFL/Master sem3/Semester Project ND/irmf/derivatives/preprocessing/sub-S01/ses-1/anat/sub-S01_ses-1_space-subject_desc-anat_bold.nii.gz')

# Create a brain mask using nilearn's masking function
brain_mask = masking.compute_brain_mask(anat_img)

# Check the shape of the brain mask
print(f"Brain mask shape: {brain_mask.shape}")

# Save the brain mask to a file (optional)
brain_mask.to_filename('brain_mask.nii.gz')


In [ ]:
fmri_img = nib.load('smoothed_fmri.nii.gz')
print(f"Shape of fMRI data: {fmri_img.shape}")

brain_mask = nib.load('brain_mask.nii.gz')
print(f"Brain mask shape: {brain_mask.shape}")


# Resample the brain mask to match the fMRI image's spatial dimensions
resampled_brain_mask = image.resample_to_img(brain_mask, fmri_img, interpolation='nearest')


print(f"Resampled brain mask shape: {resampled_brain_mask.shape}")

In [ ]:
brain_mask = nib.load('brain_mask.nii.gz')

# Load the full brain or selected voxel data
full_fmri_data = 'smoothed_fmri.nii.gz'  # This should be a 4D fMRI image

masker = NiftiMasker(mask_img=resampled_brain_mask, standardize=True)

voxel_time_series = masker.fit_transform(full_fmri_data)

# Y would be voxel_time_series here


In [8]:
voxel_time_series.shape

(528, 216949)

In [9]:
np.save('voxel_time_series.npy', voxel_time_series)

## 3) While applying ICA - to find the independent components to represent as our Y matrix

In [10]:
from sklearn.decomposition import FastICA
from nilearn.input_data import NiftiMasker

# Mask the fMRI data to focus on the brain
masker = NiftiMasker(mask_img=brain_mask, standardize=True)
#voxel_time_series = masker.fit_transform('smoothed_fmri.nii.gz')

# Apply ICA to find independent components
ica = FastICA(n_components=10)  # Number of components to retain
ica_time_series = ica.fit_transform(voxel_time_series)

# ica_time_series is your Y representing the independent components over time

/Users/shrinidhivelan/miniconda3/envs/neuroimaging/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [12]:
np.save('Y/ica_10.npy', ica_time_series)